do pliku `build.gradle(Project)`
```kotlin
plugins {
    ...
    id("com.google.dagger.hilt.android") version "2.57.2" apply false
    id("com.google.devtools.ksp") version "2.0.21-1.0.27" apply false
}
```

do pliku `build.gradle(Module)`
```kotlin
plugins {
    ...
    id("com.google.devtools.ksp")
    id("com.google.dagger.hilt.android")
}

dependencies {

    implementation("com.google.dagger:hilt-android:2.57.2")
    ksp("com.google.dagger:hilt-android-compiler:2.57.2")

    // ViewModel i Lifecycle w Compose
    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.4")
    implementation("androidx.lifecycle:lifecycle-runtime-compose:2.9.4")
    implementation("androidx.hilt:hilt-navigation-compose:1.3.0")
    ...
}
```

```kotlin

// =================================================================================
// --- Warstwa Danych: Repozytorium ---
// =================================================================================

class ProductRepository @Inject constructor() {
    // Symuluje cenę bazową produktu, np. pobraną z bazy danych lub API
    fun getBasePriceFlow(): Flow<Double> = flowOf(100.0)
}

// =================================================================================
// --- Warstwa Domeny: Use Cases ---
// =================================================================================

/**
 * Use Case #1: Oblicza cenę z podatkiem VAT. Czysta logika biznesowa.
 */
class CalculateVatPriceUseCase @Inject constructor() {
    private val vatRate = 0.23
    operator fun invoke(basePrice: Double): Double = basePrice * (1 + vatRate)
}

/**
 * Use Case #2: Sprawdza, czy kupon jest poprawny i zwraca stopę zniżki.
 */
class ValidateCouponUseCase @Inject constructor() {
    operator fun invoke(couponCode: String): Double {
        return when (couponCode.uppercase()) {
            "PROMO10" -> 0.10 // 10% zniżki
            "SUMMER20" -> 0.20 // 20% zniżki
            else -> 0.0 // Brak zniżki
        }
    }
}

/**
 * Use Case #3: Oblicza cenę po zniżce.
 */
class ApplyDiscountUseCase @Inject constructor() {
    operator fun invoke(price: Double, discountRate: Double): Double {
        return price * (1 - discountRate)
    }
}

/**
 * Use Case #4: Formatuje liczbę do stringa walutowego (logika prezentacji).
 */
class FormatCurrencyUseCase @Inject constructor() {
    private val currencyFormatter = NumberFormat.getCurrencyInstance(Locale.forLanguageTag("pl-PL"))
    operator fun invoke(value: Double): String {
        return currencyFormatter.format(value)
    }
}

// =================================================================================
// --- Architektura: ViewModel i Stan ---
// =================================================================================

data class CartUiState(
    val basePrice: String = "PLN 0,00",
    val priceWithVat: String = "PLN 0,00",
    val finalPrice: String = "PLN 0,00",
    val couponCode: String = ""
)

@HiltViewModel
class CartViewModel @Inject constructor(
    productRepository: ProductRepository,
    private val calculateVatPriceUseCase: CalculateVatPriceUseCase,
    private val validateCouponUseCase: ValidateCouponUseCase,
    private val applyDiscountUseCase: ApplyDiscountUseCase,
    private val formatCurrencyUseCase: FormatCurrencyUseCase
) : ViewModel() {

    private val couponCodeFlow = MutableStateFlow("")

    fun onCouponCodeChanged(newCode: String) {
        couponCodeFlow.value = newCode
    }

    val uiState: StateFlow<CartUiState> = combine(
        productRepository.getBasePriceFlow(),
        couponCodeFlow
    ) { basePrice, coupon ->
        // ViewModel działa jak koordynator, wywołując specjalistów (Use Cases)
        val priceWithVat = calculateVatPriceUseCase(basePrice)
        val discountRate = validateCouponUseCase(coupon)
        val finalPrice = applyDiscountUseCase(priceWithVat, discountRate)

        // Przekształcamy surowe dane na sformatowane Stringi gotowe dla UI
        CartUiState(
            basePrice = formatCurrencyUseCase(basePrice),
            priceWithVat = formatCurrencyUseCase(priceWithVat),
            finalPrice = formatCurrencyUseCase(finalPrice),
            couponCode = coupon
        )
    }.stateIn(
        scope = viewModelScope,
        started = SharingStarted.WhileSubscribed(5000),
        initialValue = CartUiState()
    )
}

// =================================================================================
// --- Konfiguracja Hilt i UI ---
// =================================================================================

@HiltAndroidApp
class MainApplication : Application()

@AndroidEntryPoint
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            DomainLayerBasicsExampleTheme {
                CartScreen()
            }
        }
    }
}

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun CartScreen(viewModel: CartViewModel = hiltViewModel()) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Podsumowanie Koszyka") }) }) { padding ->
        Column(
            modifier = Modifier
                .fillMaxSize()
                .padding(padding)
                .padding(16.dp),
            horizontalAlignment = Alignment.CenterHorizontally
        ) {
            PriceRow("Cena bazowa:", uiState.basePrice)
            PriceRow("Cena z VAT (23%):", uiState.priceWithVat)

            Spacer(Modifier.height(24.dp))

            OutlinedTextField(
                value = uiState.couponCode,
                onValueChange = { viewModel.onCouponCodeChanged(it) },
                label = { Text("Wpisz kod rabatowy (np. PROMO10)") },
                modifier = Modifier.fillMaxWidth()
            )

            Spacer(Modifier.height(32.dp))

            Text("Cena Końcowa", style = MaterialTheme.typography.headlineSmall)
            Text(
                text = uiState.finalPrice,
                style = MaterialTheme.typography.displayMedium,
                fontWeight = FontWeight.Bold,
                color = MaterialTheme.colorScheme.primary
            )
        }
    }
}

@Composable
fun PriceRow(label: String, price: String) {
    Row(
        modifier = Modifier.fillMaxWidth().padding(vertical = 8.dp),
        horizontalArrangement = Arrangement.SpaceBetween
    ) {
        Text(label, style = MaterialTheme.typography.bodyLarge)
        Text(price, style = MaterialTheme.typography.bodyLarge, fontWeight = FontWeight.SemiBold)
    }
}

```